In [72]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import sparse
from scipy.linalg import svd
from tqdm.notebook import tqdm

%matplotlib inline

### Загрузим данные

In [73]:
users = pd.read_csv('data/users.csv', low_memory=False)
organisations = pd.read_csv('data/organisations.csv', low_memory=False)
features = pd.read_csv('data/features.csv', low_memory=False)
aspects = pd.read_csv('data/aspects.csv', low_memory=False)
rubricks = pd.read_csv('data/rubrics.csv', low_memory=False)
reviews = pd.read_csv('data/reviews.csv', low_memory=False)


to_list = lambda rubrics: [int(rubric) for rubric in str(rubrics).split(' ')]


def apply_to_columns(df, columns, func=to_list):
    for column in columns:
        df.loc[~df[column].isnull(), column] = df.loc[~df[column].isnull(), column].apply(func)

columns_to_int = ['rubrics_id', 'features_id']
apply_to_columns(organisations, columns_to_int)

### Попробуем реализовать алгоритм сингулярного разложения матриц
(SVD - Singular-Value Decomposition)
Для этого сначала преобразуем данные к матричному виду, где строка - пользователи,
а столбцы - продукты, с котороыми пользователи взаимодействовали

In [74]:
aspects_columns = [id for id, aspect in aspects.values]
features_columns = [id for id, feature in features.values]
rubricks_columns = [id for id, rubrick in rubricks.values]
columns = aspects_columns + features_columns + rubricks_columns

data = users.merge(reviews, on="user_id")
data = data.rename({"city": "user_city", 'rating': 'user_rating'}, axis=1)
data = data.merge(organisations, on="org_id")
data = data.rename({"city": "org_city"}, axis=1)

for column_name in columns:
    data[column_name] = 0

print(data.shape)
print(data.dtypes.head(15))

C:\Users\A4F7~1\AppData\Local\Temp/ipykernel_9936/2746608264.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[column_name] = 0


(3640835, 228)
user_id          uint64
user_city        object
org_id           uint64
user_rating     float64
ts                int64
aspects          object
org_city         object
average_bill    float64
rating          float64
rubrics_id       object
features_id      object
1                 int64
2                 int64
3                 int64
4                 int64
dtype: object


Получили датасет размерностью (3640835, 223)
Теперь необходимо заполнить новые колонки данными

In [92]:
for row in tqdm(range(0, len(data))):
    if data.iloc[row, 5] is not np.nan:
        for aspect in data.iloc[row, 5]:
            data._set_value(row, aspect, 1)
    if data.iloc[row, 10] is not np.nan:
        for feature in data.iloc[row, 10]:
            data._set_value(row, feature, 1)
    for rubrick in data.iloc[row, 9]:
        data._set_value(row, rubrick, 1)

print(data.head(10))

  0%|          | 0/3640835 [00:00<?, ?it/s]

                user_id user_city               org_id  user_rating    ts  \
0    523295021912509756       msk  5145242920031317950          5.0   819   
1    362006428924147790       msk  5145242920031317950          5.0   839   
2  10686793557064657689       msk  5145242920031317950          5.0   975   
3   9907863350447728102       msk  5145242920031317950          5.0  1145   
4  13999966736615242178       msk  5145242920031317950          5.0  1093   
5   9199988279729242755       msk  5145242920031317950          4.0   876   
6  10533787511801499419       msk  5145242920031317950          5.0   923   
7   2065536097155165002       msk  5145242920031317950          4.0  1095   
8   9165256876644519976       msk  5145242920031317950          5.0   920   
9   8752067030694028029       msk  5145242920031317950          5.0  1001   

  aspects org_city  average_bill    rating rubrics_id  ...    0         2  \
0     NaN      msk         500.0  4.934783    [31495]  ...  NaN  NaN  NaN  

### Отбросим более не нужные колонки и заменим nan на 0

In [96]:
data = data.drop(data.columns[[5, 9, 10]], axis=1)
data = data.fillna(0)
print(data.head(10))


                user_id user_city               org_id  user_rating    ts  \
0    523295021912509756       msk  5145242920031317950          5.0   819   
1    362006428924147790       msk  5145242920031317950          5.0   839   
2  10686793557064657689       msk  5145242920031317950          5.0   975   
3   9907863350447728102       msk  5145242920031317950          5.0  1145   
4  13999966736615242178       msk  5145242920031317950          5.0  1093   
5   9199988279729242755       msk  5145242920031317950          4.0   876   
6  10533787511801499419       msk  5145242920031317950          5.0   923   
7   2065536097155165002       msk  5145242920031317950          4.0  1095   
8   9165256876644519976       msk  5145242920031317950          5.0   920   
9   8752067030694028029       msk  5145242920031317950          5.0  1001   

  org_city  average_bill    rating  1  2  ...    0         2    7    4    6  \
0      msk         500.0  4.934783  0  0  ...  0.0  0.0  0.0  0.0  0.0  0